# Task 2b: Attribute Detection (View Angle) with EfficientDet

**Student:** Luigi Camilleri  
**Student ID:** 0312805L    
**Model:** EfficientDet (D2) 


This notebook implements a detector specifically for classifying the viewing angle of traffic signs using EfficientDet. It covers:
1.  **Dataset Preparation**: Loading images and annotations.
2.  **Model Training**: Fine Tuning a pretrained `EfficientDet` to detect and classify viewing angles.
3.  **Evaluation**: Calculating F1-Scores.
4.  **Inference & Analytics**: Visualisation and counting detections of viewing angles.

In [ ]:
from __future__ import annotations

# Standard library
import importlib
import subprocess
import sys
import json
import os

# Package bootstrap helper
def ensure_package(pkg: str, import_name: str | None = None, pip_name: str | None = None):
    try:
        return importlib.import_module(import_name or pkg)
    except ImportError:
        pip_target = pip_name or pkg
        print(f"Installing missing package: {pip_target}")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", pip_target],
            stdout=subprocess.DEVNULL
        )
        return importlib.import_module(import_name or pkg)

# Ensure third-party packages
torch = ensure_package("torch")
torchvision = ensure_package("torchvision")
plt = ensure_package("matplotlib.pyplot", "matplotlib.pyplot", "matplotlib")
np = ensure_package("numpy")
PIL = ensure_package("Pillow", "PIL", "pillow")
time = ensure_package("time")
ensure_package("tensorboard")
ensure_package("albumentations")
ensure_package("pycocotools")
ensure_package("effdet")
ensure_package("pandas")

# --- Explicit imports (clear & IDE-friendly) ---
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T
from PIL import Image
import time
import copy
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pycocotools.coco import COCO
import pandas as pd

# Custom/local utilities 
try:
    from luigi_camilleri_utils import *
except ImportError:
    raise ImportError(
        "❌ 'luigi_camilleri_utils' not found.\n"
        "Make sure 'luigi_camilleri_utils.py' is in the working directory "
        "or on PYTHONPATH."
    )

# Reproducibility
torch.manual_seed(42)

print("Environment ready: all dependencies installed and imported.")

## 1. Dataset Preparation


I use the `SignsDataset` class from my utilities module.


**Note:** Images are stored in `images` subfolder and annotations in `annotations` subfolder. These two subfolders are found in the `COCO-based_COCO_view_angle` folder.

In each subfolder there are the data split into `train`, `val` and `test`. This is done to make sure that the results are reproducible.

In [ ]:
DATA_DIR_TRAIN = "Assignment Material/COCO-based_COCO_view_angle/images/train" 
ANNOTATION_FILE_TRAIN = "Assignment Material/COCO-based_COCO_view_angle/annotations/train.json"

DATA_DIR_VAL = "Assignment Material/COCO-based_COCO_view_angle/images/val"
ANNOTATION_FILE_VAL = "Assignment Material/COCO-based_COCO_view_angle/annotations/val.json"

The categories are also taken from the annotations file, this ensures that the categories are consistent. Additionally it avoids having mistakes or having to change the category list if the attribute type is changed.

In [ ]:
# Build mapping: class_id -> class_name
with open(ANNOTATION_FILE_TRAIN, "r") as f:
    data = json.load(f)
CLASS_ID_TO_NAME = {cat["id"]: cat["name"] for cat in data["categories"]}
NUM_CLASSES = len(CLASS_ID_TO_NAME)  
# print(CLASS_ID_TO_NAME)

### Initialize Dataset and DataLoaders

In [ ]:
# Data augmentation flag
augmentation = True
transform = A.Compose(
    [
        A.Resize(768, 768),  # EfficientDet expects fixed-size inputs
        A.Normalize(
            mean=(0.0, 0.0, 0.0),
            std=(1.0, 1.0, 1.0),
            max_pixel_value=255.0
        ),
        ToTensorV2()
    ],
    bbox_params=A.BboxParams(
        format="coco",
        label_fields=["labels"]
    )
)

if augmentation:
    train_transform = A.Compose(
    [
        A.Resize(768, 768),  # EfficientDet expects fixed-size inputs
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.Affine(
            scale=(0.9, 1.1),
            translate_percent=(0.05, 0.05),
            rotate=(-15, 15),
            mode=0,
            p=0.5
        ),
        A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
        ToTensorV2()
    ],
    bbox_params=A.BboxParams(format="coco", label_fields=["labels"])
)
else:
    train_transform = transform

train_dataset = SignsDataset(
    root=DATA_DIR_TRAIN,
    annFile=ANNOTATION_FILE_TRAIN,
    transforms=train_transform,
)

val_dataset = SignsDataset(
    root=DATA_DIR_VAL,
    annFile=ANNOTATION_FILE_VAL,
    transforms=transform, 
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=4, 
    shuffle=True, 
    collate_fn=lambda x: tuple(zip(*x)))

val_loader = DataLoader(
    val_dataset, 
    batch_size=2, 
    shuffle=False, 
    collate_fn=lambda x: tuple(zip(*x)))

## 2. Model Configuration
Loading the EfficientDet model pre-trained on COCO and adapting proper number of classes.

In [ ]:
device = get_device()
model, config = get_efficientdet_train(NUM_CLASSES)
model = model.to(device)

lr = 0.001
weight_decay = 1e-4
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=lr,
    weight_decay=weight_decay
)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=3, gamma=0.1
)

## 3. Training Loop

Training for 25 epochs and evaluating on validation set after each epoch.

In [ ]:
num_epochs = 25
train_losses = []
val_f1_scores = []
train_cls_losses = []
train_box_losses = []
best_epochs = 0
best_f1 = 0
best_model = None
writer = SummaryWriter("runs/efficientdet_view_angle")

start_time = time.time()
for epoch in range(num_epochs):
    print(f"\nStarting Epoch {epoch+1}/{num_epochs}")
    losses = train_one_epoch(model, train_loader, optimizer, device)
    
    train_losses.append(losses["total"])
    train_cls_losses.append(losses["cls"])
    train_box_losses.append(losses["box"])

    writer.add_scalar("Loss/Total", losses["total"], epoch)
    writer.add_scalar("Loss/Class", losses["cls"], epoch)
    writer.add_scalar("Loss/Box", losses["box"], epoch)

    scheduler.step()

    f1 = f1_score_by_iou(model, val_loader, device, config)
    val_f1_scores.append(f1)

    if f1 > best_f1:
        best_f1 = f1
        best_model = copy.deepcopy(model)
        best_epochs = epoch + 1

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Total Loss: {losses['total']:.4f} | "
        f"Cls Loss: {losses['cls']:.4f} | "
        f"Box Loss: {losses['box']:.4f} | "
        f"F1: {f1:.4f}"
    )

writer.close()
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds")
print(f"Best F1 score: {best_f1:.4f} at epoch {best_epochs}")

### Reload the best model only if needed

In [ ]:
reload = False
if reload:
    best_model, config = get_efficientdet_train(NUM_CLASSES)
    best_model = best_model.to(device)
    state_dict = torch.load(
    'models/augmentation/efficientdet_epochs25_lr_0.0001_weight_0.0001_view_angle.pt',
     map_location=device
    )
    best_model.load_state_dict(state_dict)
    model = best_model.to(device)

### MAP Score

In [ ]:
coco_gt = COCO(ANNOTATION_FILE_VAL)
map_stats = evaluate_map(best_model, val_loader, device, coco_gt, config)

metrics_names = [
    "mAP IoU=0.5:0.95", "mAP IoU=0.5", "mAP IoU=0.75",
    "mAP small", "mAP medium", "mAP large",
    "AR max=1", "AR max=10", "AR max=100",
    "AR small", "AR medium", "AR large",
    "F1@IoU=0.5"
]

metrics_values = list(map_stats) + [0.001]

metrics_df = pd.DataFrame({
    "Metric": metrics_names,
    "Value": metrics_values
})

print(metrics_df)
os.makedirs("runs/efficientdet_view_angle", exist_ok=True)
metrics_df.to_csv("runs/efficientdet_view_angle/best_model_full_metrics.csv", index=False)

### Save the model

In [ ]:
os.makedirs("./models", exist_ok=True)
final = False
if final:
    os.makedirs("./models/efficientdet_view_angle", exist_ok=True)
    torch.save(best_model.state_dict(), f"./models/efficientdet_view_angle/FINAL_epochs{num_epochs}_lr_{lr}_weight_{weight_decay}_view_angle.pt")
elif augmentation:
    os.makedirs("./models/augmentation", exist_ok=True)
    torch.save(best_model.state_dict(), f"./models/augmentation/efficientdet_epochs{num_epochs}_lr_{lr}_weight_{weight_decay}_view_angle.pt")
else:
    torch.save(best_model.state_dict(), f"./models/efficientdet_epochs{num_epochs}_lr_{lr}_weight_{weight_decay}_view_angle.pt")

### Plotting the losses for the view angle

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(range(1, num_epochs + 1), train_box_losses,
         label='Train Box Loss', marker='o')

plt.plot(range(1, num_epochs + 1), train_cls_losses,
         label='Train Classification Loss', marker='o')

plt.title("Training Losses Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

### Plotting the F1-score for each epoch

In [ ]:
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(10, 6))
plt.plot(
    epochs,
    val_f1_scores,
    marker="o",
    linestyle="-",
    label="Validation F1-score"
)

plt.xlabel("Epoch")
plt.ylabel("F1-score")
plt.title("Validation F1-score vs Epochs")
plt.grid(True)
plt.legend()
plt.show()

## 4. Analytics & Visualization

Analyze the detections on the validation set. I will count the number of detected signs and visualize the results. I apply inference on the validation set which are unlabelled unseen examples.

### Plot of F1-Score vs Confidence Threshold

In [ ]:
confidence_thresholds = np.linspace(0.1, 0.95, 18)

f1_val_scores = []
f1_threshold = 0.0
best_thresh = 0.0
for thresh in confidence_thresholds:
    f1 = f1_score_by_iou(best_model, val_loader, device, config, iou_threshold=0.5, score_threshold=thresh)
    f1_val_scores.append(f1)
    print(f"Threshold: {thresh:.2f}, F1 Score: {f1:.4f}")
    if f1 > f1_threshold:
        f1_threshold = f1
        best_thresh = thresh

plt.figure(figsize=(8,5))
plt.plot(confidence_thresholds, f1_val_scores, marker='o')
plt.title("F1 Score vs Confidence Threshold")
plt.xlabel("Confidence Threshold")
plt.ylabel("F1 Score")
plt.grid(True)
plt.show()

In [ ]:
from effdet import DetBenchPredict

eval_model = DetBenchPredict(best_model.model)
eval_model.to(device)
eval_model.eval()

total_detected_signs = 0
results_summary = []

with torch.no_grad():
    for img, target in val_dataset:
        img_batch = img.unsqueeze(0).to(device)
        prediction = eval_model(img_batch)[0]
        
        # EfficientDet output: [num_dets, 6] -> [x1, y1, x2, y2, score, class]
        keep = prediction[:, 4] > best_thresh
        final_dets = prediction[keep]
        
        count = len(final_dets)
        total_detected_signs += count
        
        results_summary.append({
            "ImageID": target["image_id"].item(),
            "DetectedSigns": count,
            "Labels": [CLASS_ID_TO_NAME.get(int(det[5].item()), "Unknown") for det in final_dets],
            "Labelids": [int(det[5].item()) for det in final_dets],
            "Scores": [det[4].item() for det in final_dets],
            "Boxes": [det[:4].tolist() for det in final_dets]
        })

print(f"Total signs detected in validation set: {total_detected_signs}")
print("Results summary:")
for res in results_summary:
    print(res)

# Save results to a text file
os.makedirs("./results", exist_ok=True)
with open(f"./results/results_view_angle_epochs{num_epochs}_threshold{best_thresh}.txt", "w") as f:
    for res in results_summary:
        f.write(f"{res}\n")
    f.write(f"Total detected signs: {total_detected_signs}")

### Qualitative results and Visualisation

In [ ]:
print("\nVisualizing Sample Predictions:")
for i in range(len(val_dataset)):
    img, target = val_dataset[i]
    with torch.no_grad():
        img_batch = img.unsqueeze(0).to(device)
        prediction = eval_model(img_batch)[0]
    
    gt_labels = [CLASS_ID_TO_NAME.get(l.item(), "Unknown") for l in target['labels']]
    print(f"Sample {i+1} Ground Truth: {gt_labels}")
    visualize_predictions(img, prediction, CLASS_ID_TO_NAME, threshold=best_thresh)

## Clean up memory

In [ ]:
import gc
import torch
import matplotlib.pyplot as plt

plt.close("all")
torch.cuda.empty_cache()
gc.collect()